## Notebook to combine all feature engineered data and merge it with all non-feature engineered data

In [1]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
from functools import reduce

In [2]:
for i in range (2003, 2020):
    
    year=str(i)
    
    PATH_1=os.path.join('..','all_years','CAMS', year ,'feature_engineering')

    # list all the pkl files in the folder
    pkl_files = [file for file in os.listdir(PATH_1) if file.endswith('.pkl')]

    ## read all the pickle files
    dataframes = [pd.read_pickle(os.path.join(PATH_1, file)) for file in pkl_files]

    ## remove the NAME columns from the dataframes. This step was necessary because, functools (below) didnt work
    ## when dataframes had same columns

    for j in range(len(dataframes)):
        if "NAME" in dataframes[j].columns:
            dataframes[j] = dataframes[j].drop(columns="NAME")
   
    #from functools import reduce. merge all the datframes in the list on fips
    df1=reduce(lambda x, y: pd.merge(x, y, on = 'fips'), dataframes)
    df1

    ## get the pm25 above threshold data which is on a different folder
    PATH_2=os.path.join('..','all_years','CAMS', year, 'FE_PM25_'+ year +'.pkl')
    df2=pd.read_pickle(PATH_2)
    df2 = df2.drop('NAME', axis=1)
    df2

    final_df=df1.merge(df2,how='left', on='fips')
    final_df['year']=int(i)
    final_df
    
    ## save in the feature_engineered file
    PATH_3='feature_engineered'
    file=os.path.join(PATH_3, 'fe_'+ year +'.pkl')
    final_df.to_pickle(file)


In [3]:
 # list all the feature engineered data
PATH_4=os.path.join('feature_engineered')
pkl_files = [file for file in os.listdir(PATH_4) if file.endswith('.pkl')]

## read all the pickle files
dataframes = [pd.read_pickle(os.path.join(PATH_4, file)) for file in pkl_files]

In [4]:
df_concat=pd.concat(dataframes)
df_concat = df_concat.drop(['ALAND', 'AWATER','INTPTLAT','INTPTLON'], axis=1) ## not sure how these appeared
df_concat

,fips,co above percentile,ethane above percentile,formaldehyde above percentile,hydroxyl above percentile,isoprene above percentile,nitrate above percentile,nitric above percentile,nitrogen dioxide above percentile,Nitrogen above percentile,ozone above percentile,peroxide above percentile,PM10 above percentile,PM10 above threshold,PM1 above percentile,PM2.5 above percentile,propane above percentile,so2 above percentile,Temp above percentile,Temp above threshold,Temp below percentile,Temp below threshold,PM2.5 above threshold,year
0,19175,25.513698630136989,20.821917808219180,14.931506849315069,23.390410958904109,3.116438356164384,12.979452054794521,24.041095890410961,14.554794520547945,9.246575342465754,35.753424657534246,23.356164383561644,28.287671232876711,0.000000000000000,29.178082191780824,27.671232876712327,5.273972602739726,0.000000000000000,38.150684931506852,0.993150684931507,43.664383561643838,23.356164383561644,70.308219178082183,2003
1,19177,30.445205479452053,27.020547945205482,20.650684931506849,20.821917808219180,7.773972602739726,21.130136986301370,24.452054794520546,21.164383561643834,12.328767123287671,35.410958904109592,21.164383561643834,36.643835616438359,0.000000000000000,38.390410958904106,36.232876712328768,10.410958904109590,6.952054794520548,38.184931506849317,0.684931506849315,42.534246575342465,21.472602739726028,75.513698630136986,2003
2,20097,6.780821917808219,9.965753424657533,9.897260273972602,31.678082191780820,0.000000000000000,7.602739726027398,18.801369863013697,1.815068493150685,1.849315068493151,43.390410958904106,30.856164383561648,7.945205479452055,0.000000000000000,7.773972602739726,8.219178082191780,0.479452054794521,0.000000000000000,45.308219178082190,3.938356164383562,35.753424657534246,13.390410958904109,43.595890410958901,2003
3,20109,2.054794520547945,4.726027397260274,2.773972602739726,31.198630136986299,0.000000000000000,1.712328767123288,14.075342465753424,0.821917808219178,1.335616438356164,46.678082191780824,29.075342465753423,4.280821917808219,0.000000000000000,3.972602739726028,4.383561643835616,0.000000000000000,0.000000000000000,40.308219178082197,2.465753424657534,40.787671232876711,16.952054794520549,36.541095890410958,2003
4,20003,22.602739726027394,25.582191780821915,23.527397260273972,25.000000000000000,5.753424657534246,24.075342465753423,26.678082191780820,21.267123287671232,12.671232876712329,40.239726027397261,27.568493150684930,26.541095890410958,0.856164383561644,27.157534246575345,26.335616438356162,10.616438356164384,19.691780821917810,45.958904109589042,2.671232876712329,35.958904109589042,13.458904109589040,71.061643835616437,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,20119,15.239726027397261,25.547945205479451,5.890410958904110,33.698630136986303,0.000000000000000,3.287671232876712,32.842465753424662,5.547945205479452,8.116438356164384,41.541095890410958,31.917808219178085,18.253424657534246,0.000000000000000,16.335616438356166,18.321917808219180,0.650684931506849,0.000000000000000,42.979452054794521,3.116438356164384,37.191780821917810,12.979452054794521,30.856164383561648,2019
3229,38049,21.643835616438356,30.958904109589042,0.136986301369863,21.438356164383563,0.000000000000000,0.205479452054795,13.253424657534246,18.698630136986303,20.102739726027398,26.643835616438356,10.616438356164384,25.205479452054796,0.000000000000000,27.123287671232877,24.623287671232877,5.547945205479452,15.410958904109590,21.061643835616440,0.000000000000000,59.589041095890416,39.280821917808218,40.410958904109592,2019
3230,72119,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000

In [5]:
df_previous=pd.read_pickle('complete_dataset.pkl')
df_previous

,fips,year,MeanLifeExpectency,2m dew point temperature,2m temperature,Black carbon AOD at 550 nm,Dust AOD at 550 nm,Land-sea mask,Mean sea level pressure,Organic matter AOD at 550 nm,PM$_1$,PM$_{2.5}$,PM$_{10}$,Sea salt AOD at 550 nm,Sulphate AOD at 550 nm,Surface geopotential,Surface pressure,Total AOD at 469 nm,Total AOD at 550 nm,Total AOD at 670 nm,Total AOD at 865 nm,Total AOD at 1240 nm,Total column carbon monoxide,Total column ethane,Total column formaldehyde,Total column hydrogen peroxide,Total column hydroxyl radical,Total column isoprene,Total column methane,Total column nitric acid,Total column nitrogen dioxide,Total column nitrogen monoxide,Total column ozone,Total column peroxyacetyl nitrate,Total column propane,Total column sulphur dioxide,Total column water vapour,Carbon monoxide,Dust aerosol (0.03-0.55 µm) mixing ratio,Dust aerosol (0.55-0.9 µm) mixing ratio,Dust aerosol (0.9-20 µm) mixing ratio,Ethane,Formaldehyde,Hydrogen peroxide,Hydrophilic black carbon aerosol mixing ratio,Hydrophilic organic matter aerosol mixing ratio,Hydrophobic black carbon aerosol mixing ratio,Hydrophobic organic matter aerosol mixing ratio,Hydroxyl radical,Isoprene,Nitric acid,Nitrogen dioxide,Nitrogen monoxide,Ozone,Peroxyacetyl nitrate,Propane,Sea salt aerosol (0.03-0.5 µm) mixing ratio,Sea salt aerosol (0.5-5 µm) mixing ratio,Sea salt aerosol (5-20 µm) mixing ratio,Specific humidity,Sulphate aerosol mixing ratio,Sulphur dioxide,Temperature,"Leaf area index, high vegetation","Leaf area index, low vegetation",Snow albedo,Snow depth,Total column water,Vertically integrated mass of dust aerosol (0.03-0.55 µm),Vertically integrated mass of dust aerosol (0.55-9 µm),Vertically integrated mass of dust aerosol (9-20 µm),Vertically integrated mass of hydrophilic black carbon aerosol,Vertically integrated mass of hydrophilic organic matter aerosol,Vertically integrated mass of hydrophobic black carbon aerosol,Vertically integrated mass of hydrophobic organic matter aerosol,Vertically integrated mass of sea salt aerosol (0.03-0.5 µm),Vertically integrated mass of sea salt aerosol (0.5-5 µm),Vertically integrated mass of sea salt aerosol (5-20 µm),Vertically integrated mass of sulphate aerosol,Vertically integrated mass of sulphur dioxide,Relative humidity,Specific humidity,10m wind speed,Wet bulb temperature
0,1001,2003,74.628765329999993,285.629269414872169,290.863352214209158,0.007571248154860,0.010371692890225,0.986464531564209,101746.460785242583370,0.097522246908886,0.000000015615338,0.000000019601377,0.000000027192044,0.008606192940150,0.112170314070482,1368.670051439976305,100086.817007783130975,0.288798537560071,0.236241944078782,0.180380990078313,0.123618863685572,0.069164993038643,0.000984550381998,0.000003862356441,0.000003599970514,0.000008836077454,0.000000004853489,0.000001984446001,0.009753834615672,0.000007246513254,0.000004130302193,0.000000542480041,0.006426156201957,0.000008713431766,0.000000740202970,0.000004737026637,25.902900677695214,0.000000230284119,0.000000000654884,0.000000001134462,0.000000000403961,0.000000000705993,0.000000003513651,0.000000000741261,0.000000000224510,0.000000010096395,0.000000000254529,0.000000005847693,0.000000000000028,0.000000014604292,0.000000001474772,0.000000010235872,0.000000000867748,0.000000058145540,0.000000003056157,0.000000000328437,0.000000000023239,0.000000001862196,0.000000000212949,0.010010040534155,0.000000004044040,0.000000009229008,17.605204629052423,3.904460199286052,2.618741604547198,0.879750469058084,0.000000093784908,26.035876613510123,0.000001993471045,0.000004062559380,0.000003433003025,0.000000147754324,0.000003227288298,0.000000653503172,0.000015866768387,0.000000220848889,0.000017570806448,0.000000962454028,0.000011681476149,0.000002886328880,71.152047054515833,0.010074853188795,0.209342152896552,14.185547266162992
1,1003,2003,76.661419230000007,289.075082204243699,292.904504754344714,0.007386436848696,0.012786048677104,0.422456727373741,101714.385671383512090,0.08053

In [6]:
complete_df=df_previous.merge(df_concat, on=['fips','year'])
complete_df

,fips,year,MeanLifeExpectency,2m dew point temperature,2m temperature,Black carbon AOD at 550 nm,Dust AOD at 550 nm,Land-sea mask,Mean sea level pressure,Organic matter AOD at 550 nm,PM$_1$,PM$_{2.5}$,PM$_{10}$,Sea salt AOD at 550 nm,Sulphate AOD at 550 nm,Surface geopotential,Surface pressure,Total AOD at 469 nm,Total AOD at 550 nm,Total AOD at 670 nm,Total AOD at 865 nm,Total AOD at 1240 nm,Total column carbon monoxide,Total column ethane,Total column formaldehyde,Total column hydrogen peroxide,Total column hydroxyl radical,Total column isoprene,Total column methane,Total column nitric acid,Total column nitrogen dioxide,Total column nitrogen monoxide,Total column ozone,Total column peroxyacetyl nitrate,Total column propane,Total column sulphur dioxide,Total column water vapour,Carbon monoxide,Dust aerosol (0.03-0.55 µm) mixing ratio,Dust aerosol (0.55-0.9 µm) mixing ratio,Dust aerosol (0.9-20 µm) mixing ratio,Ethane,Formaldehyde,Hydrogen peroxide,Hydrophilic black carbon aerosol mixing ratio,Hydrophilic organic matter aerosol mixing ratio,Hydrophobic black carbon aerosol mixing ratio,Hydrophobic organic matter aerosol mixing ratio,Hydroxyl radical,Isoprene,Nitric acid,Nitrogen dioxide,Nitrogen monoxide,Ozone,Peroxyacetyl nitrate,Propane,Sea salt aerosol (0.03-0.5 µm) mixing ratio,Sea salt aerosol (0.5-5 µm) mixing ratio,Sea salt aerosol (5-20 µm) mixing ratio,Specific humidity,Sulphate aerosol mixing ratio,Sulphur dioxide,Temperature,"Leaf area index, high vegetation","Leaf area index, low vegetation",Snow albedo,Snow depth,Total column water,Vertically integrated mass of dust aerosol (0.03-0.55 µm),Vertically integrated mass of dust aerosol (0.55-9 µm),Vertically integrated mass of dust aerosol (9-20 µm),Vertically integrated mass of hydrophilic black carbon aerosol,Vertically integrated mass of hydrophilic organic matter aerosol,Vertically integrated mass of hydrophobic black carbon aerosol,Vertically integrated mass of hydrophobic organic matter aerosol,Vertically integrated mass of sea salt aerosol (0.03-0.5 µm),Vertically integrated mass of sea salt aerosol (0.5-5 µm),Vertically integrated mass of sea salt aerosol (5-20 µm),Vertically integrated mass of sulphate aerosol,Vertically integrated mass of sulphur dioxide,Relative humidity,Specific humidity,10m wind speed,Wet bulb temperature,co above percentile,ethane above percentile,formaldehyde above percentile,hydroxyl above percentile,isoprene above percentile,nitrate above percentile,nitric above percentile,nitrogen dioxide above percentile,Nitrogen above percentile,ozone above percentile,peroxide above percentile,PM10 above percentile,PM10 above threshold,PM1 above percentile,PM2.5 above percentile,propane above percentile,so2 above percentile,Temp above percentile,Temp above threshold,Temp below percentile,Temp below threshold,PM2.5 above threshold
0,1001,2003,74.628765329999993,285.629269414872169,290.863352214209158,0.007571248154860,0.010371692890225,0.986464531564209,101746.460785242583370,0.097522246908886,0.000000015615338,0.000000019601377,0.000000027192044,0.008606192940150,0.112170314070482,1368.670051439976305,100086.817007783130975,0.288798537560071,0.236241944078782,0.180380990078313,0.123618863685572,0.069164993038643,0.000984550381998,0.000003862356441,0.000003599970514,0.000008836077454,0.000000004853489,0.000001984446001,0.009753834615672,0.000007246513254,0.000004130302193,0.000000542480041,0.006426156201957,0.000008713431766,0.000000740202970,0.000004737026637,25.902900677695214,0.000000230284119,0.000000000654884,0.000000001134462,0.000000000403961,0.000000000705993,0.000000003513651,0.000000000741261,0.000000000224510,0.000000010096395,0.000000000254529,0.000000005847693,0.000000000000028,0.000000014604292,0.000000001474772,0.000000010235872,0.000000000867748,0.000000058145540,0.000000003056157,0.000000000328437,0.000000000023239,0.000000001862196,0.000000000212949,0.010010040534155,0.000000004044040,0.000000009229008,17.605204629052423,3.904460199

In [7]:
complete_df.to_pickle('final_dataset_103_features.pkl')